In [ ]:
import numpy as np

In [70]:
# ABSOLUTE POSITIONAL EMBEDDING

def abs_positional_encoding(length, dimensions):
  # 'length' is the length of sequences
  # 'dimensions' is the dimension of input embedding

  def single_pos_embed(position): #calculating for a certain position
      return [position / np.power(10000, 2 * (i // 2) / dimensions)
              for i in range(dimensions)]

  pos_enc = np.array([single_pos_embed(i) for i in range(length)]) #adding embedding for all positions
  pos_enc[:, 0::2] = np.sin(pos_enc[:, 0::2])  # dim 2i application
  pos_enc[:, 1::2] = np.cos(pos_enc[:, 1::2])  # dim 2i+1 application
  return pos_enc

abs_pe = abs_positional_encoding(length = 10,dimensions = 5)
print(f"shape of abs_pe : {abs_pe.shape}\n")
np.set_printoptions(precision=4)
print(abs_pe)

shape of abs_pe : (10, 5)

[[ 0.0000e+00  1.0000e+00  0.0000e+00  1.0000e+00  0.0000e+00]
 [ 8.4147e-01  5.4030e-01  2.5116e-02  9.9968e-01  6.3096e-04]
 [ 9.0930e-01 -4.1615e-01  5.0217e-02  9.9874e-01  1.2619e-03]
 [ 1.4112e-01 -9.8999e-01  7.5285e-02  9.9716e-01  1.8929e-03]
 [-7.5680e-01 -6.5364e-01  1.0031e-01  9.9496e-01  2.5238e-03]
 [-9.5892e-01  2.8366e-01  1.2526e-01  9.9212e-01  3.1548e-03]
 [-2.7942e-01  9.6017e-01  1.5014e-01  9.8866e-01  3.7857e-03]
 [ 6.5699e-01  7.5390e-01  1.7493e-01  9.8458e-01  4.4167e-03]
 [ 9.8936e-01 -1.4550e-01  1.9960e-01  9.7988e-01  5.0476e-03]
 [ 4.1212e-01 -9.1113e-01  2.2415e-01  9.7455e-01  5.6786e-03]]


In [86]:
# RELATIVE POSITIONAL EMBEDDING

def rel_positional_encoding(length,dimensions,calc = 'simple',clip = None):
  # 'length' is the length of sequences
  # 'dimensions' is the dimension of input embedding
  # 'calc' decides the type of relative positional embedding to be used.
  # 'calc' = { 'simple' : returns the relative position value itself ..-3,-2,-1,0,1,2,3...
  #            'sin'    : return the sinusoidal absolute embedding related to relative position (assumes symmetric relation forward and backward)
  #          }
  # 'clip' decides after what relative position should it maintain the last relational embedding for the rest.
  #        for e.g.: if clip = 3, then for the 6th word, 'calc' = 'simple' gives -3,-3,-3,-2,-1,0,1,2,3,3,3..



  # depending upon 'calc' value, deciding the relative embedding calculations

  if calc == "simple":

    def get_w_relativepos(rel): # function that returns a simple relative positional embedding

      if clip and abs(rel) > clip: # clipping if specified
        return clip * (rel / abs(rel))
      else: return rel

  elif calc == "sin":

    def single_pos_embed(rel_position): # function that returns a sinusoidal positional embedding for a single relative position of a word w.r.t another

      embed = []
      for i in range(dimensions):
        if i % 2 == 0: embed.append(np.sin(rel_position / np.power(10000, 2 * (i // 2) / dimensions)))
        else: embed.append(np.cos(rel_position / np.power(10000, 2 * (i // 2) / dimensions)))
      return embed
    # above function can also be replaced with an array containing all possible positional embedding so the repitive calculations can be avoided


    def get_w_relativepos(rel):

      # index_sin = rel + length
      index_sin = rel

      if clip and abs(rel) > clip:
        # index_sin = (clip * (rel / abs(rel))) + length
        index_sin = clip * (rel / abs(rel))

      return single_pos_embed(index_sin)

  else:
    print("Invalid Calculation type! exited!")
    return

  result = []
  for i in range(length): # generationg relative pos embeddings for all positions
    result.append([get_w_relativepos(j-i) for j in range(length)]) # generating and adding relative pos embeddings for a word at certain position w.r.t to all words

  return np.array(result)



print(f"\t******* 'simple' relative positonal embedding ********")
rel_pe = rel_positional_encoding(length = 10, dimensions = 5, calc = "simple", clip = 0)
print(f"shape of rel_pe : {rel_pe.shape}\n")
np.set_printoptions(precision=4)
print(f"The relative embedding a_(0,0) : \n{rel_pe[0][0]}")
print(f"The relative embedding a_(1,1) : \n{rel_pe[1][1]}\n")
print(f"The relative embedding a_(1,2) : \n{rel_pe[1][2]}")
print(f"The relative embedding a_(1,0) : \n{rel_pe[1][0]}")


print(f"\n\n\n\t******* 'sin' relative positonal embedding ********")
rel_pe = rel_positional_encoding(length = 10, dimensions = 5, calc = "sin", clip = 0)
print(f"shape of rel_pe : {rel_pe.shape}\n")
np.set_printoptions(precision=4)
print(f"The relative embedding a_(0,0) : \n{rel_pe[0][0]}")
print(f"The relative embedding a_(1,1) : \n{rel_pe[1][1]}\n")
print(f"The relative embedding a_(1,2) : \n{rel_pe[1][2]}")
print(f"The relative embedding a_(1,0) : \n{rel_pe[1][0]}")

	******* 'simple' relative positonal embedding ********
shape of rel_pe : (10, 10)

The relative embedding a_(0,0) : 
0
The relative embedding a_(1,1) : 
0

The relative embedding a_(1,2) : 
1
The relative embedding a_(1,0) : 
-1



	******* 'sin' relative positonal embedding ********
shape of rel_pe : (10, 10, 5)

The relative embedding a_(0,0) : 
[0. 1. 0. 1. 0.]
The relative embedding a_(1,1) : 
[0. 1. 0. 1. 0.]

The relative embedding a_(1,2) : 
[8.4147e-01 5.4030e-01 2.5116e-02 9.9968e-01 6.3096e-04]
The relative embedding a_(1,0) : 
[-8.4147e-01  5.4030e-01 -2.5116e-02  9.9968e-01 -6.3096e-04]


In [84]:
# ROTARY POSITIONAL EMBEDDINGS

def rot_positional_encoding(length, dimensions):

  thetas = [10000 ** (-2 * (i - 1) / dimensions) for i in range(int(dimensions / 2))] # FInding the fixed theta values

  # genrating a single rotational matrix for a certain position
  def gen_rotary_pos_embed_matrix(position):
    rot_matrix = np.zeros((dimensions, dimensions))
    for i in range(int(dimensions / 2)):
      rot_matrix[(2*i)][(2*i)] =  np.cos(position * thetas[i])
      rot_matrix[(2*i)][(2*i)+1] = - np.sin(position * thetas[i])
      rot_matrix[(2*i)+1][(2*i)] =  np.sin(position * thetas[i])
      rot_matrix[(2*i)+1][(2*i)+1] =  np.cos(position * thetas[i])
    return rot_matrix


  output = []
  for i in range(length): # calculating the rotational matrices for all positions
     output.append(gen_rotary_pos_embed_matrix(i))

  return np.array(output)



rot_pe = rot_positional_encoding(length = 10, dimensions = 4)
print(f"shape of rot_pe : {rot_pe.shape}\n")
np.set_printoptions(precision=4)
print(f"rotational matrix for first word :")
print(rot_pe[0])

shape of rot_pe : (10, 4, 4)

rotational matrix for first word :
[[ 1. -0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1. -0.]
 [ 0.  0.  0.  1.]]
